In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import random
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
X = [[0, 0], [1, 1]]
y = [0, 1]

modelo = SVC()
modelo.fit(X,y)

In [ ]:
modelo.predict([[0,0],[1,0],[0,1],[1,1],[0.49,0.49]])

In [ ]:
# Descobrir caminhos de imagens e classificação
images_path = '../dataset_converted'

# loaded variables
df = pd.DataFrame(columns=['path','classification'])
classification_label = []

for f_number, folder in enumerate(os.listdir(images_path)):
    folder_path = os.path.join(images_path, folder)
    classification_label.append(folder)
    for image in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image)
        df.loc[0 if pd.isnull(df.index.max()) else df.index.max() + 1] = [image_path, f_number]

classification_label, df

In [ ]:
# Carregar as imagens e fazer pré-processamento
def preprocess_image(image_path):
   # Carregar a imagem em escala de cinza
   image = np.array(Image.open(image_path).convert('L'))
   # Aplicar equalização de histograma
   # image = cv2.equalizeHist(image)
   return image.ravel()

In [ ]:
# Plotar a imagem original e a equalizada
def plot_images(equalized):
    plt.figure(figsize=(10, 5))
    # Mostrar imagem equalizada
    plt.subplot()
    plt.title("Equalized Image")
    plt.imshow(equalized, cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
# Pré-processar a imagem
equalized_image = preprocess_image(df['path'][100])

In [ ]:
# Plotar as imagens
plot_images(equalized_image.reshape(100,100))

In [ ]:
df['image'] = df['path'].apply(preprocess_image)
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['image'], df['classification'], test_size=0.2, random_state=42)

df_train = pd.concat([X_train, y_train], axis=1)

X_train = np.vstack(X_train.values)
y_train = y_train.values
X_test = np.vstack(X_test.values)
y_test = y_test.values

df_train, X_test, y_test, len(X_test), len(y_test)

In [ ]:
class_counts = df_train['classification'].value_counts()
min_class_count = class_counts.min()
class_counts, min_class_count

# Treinar o modelo SVC

In [ ]:
modelo = SVC(max_iter=500,tol=0.000001)

In [ ]:
def train_model(model, df_train, iter_p_roll=100, max_iter=1000, class_col='classification', obj_col='image'):
    class_counts = df_train['classification'].value_counts()
    min_class_count = class_counts.min()
    
    modelo.max_iter = iter_p_roll
    n_epoch = max_iter//iter_p_roll
    
    for _ in range(n_epoch):
        balanced_df = pd.concat([
            df_train[df_train['classification'] == cls].sample(min_class_count)
            for cls in range(len(class_counts))
        ])
        balanced_df['tmp'] = balanced_df.apply(lambda x: random.random(), axis=1)
        balanced_df = balanced_df.sort_values('tmp')
        balanced_df = balanced_df.drop('tmp', axis=1)
    
        X = np.vstack(balanced_df['image'].values)
        y = balanced_df['classification'].values
        X,y
        
        model.fit(X,y)

In [ ]:
train_model(modelo, df_train, iter_p_roll=500, max_iter=20000)

In [ ]:
modelo.fit(X_train, y_train)

In [ ]:
modelo.score(X_test, y_test)

In [ ]:
df['predict'] = df['image'].apply(lambda x: modelo.predict([x])[0])
correct = df.apply(lambda x: x['classification'] == x['predict'], axis=1)
df[correct]

In [ ]:
import joblib
joblib.dump(modelo, 'modelo_svc.pkl')

In [ ]:
modelo_carregado = joblib.load('modelo_svc.pkl')
modelo_carregado.score(X_test, y_test)

In [ ]:
df['predict'] = df['image'].apply(lambda x: modelo_carregado.predict([x])[0])
correct = df.apply(lambda x: x['classification'] == x['predict'], axis=1)
df[correct]